# Code to make meta features

In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import matplotlib.pylab as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import warnings
from datetime import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)
import logging
import gc
from timeit import default_timer as timer
import time
from catboost import CatBoostRegressor, Pool
from sklearn.neighbors import KNeighborsClassifier

## Meta Function

In [2]:
FEATURES = [
            '10th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '10th_closest_to_0_valence_x_cube_inv_dist',
             '10th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '10th_closest_to_1_valence_x_cube_inv_dist',
             '12th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '12th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '13th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '14th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '21st_closest_to_0_atomic_mass_x_cube_inv_dist',
             '2nd_closest_to_0_atomic_mass',
             '2nd_closest_to_0_atomic_mass_x_cube_inv_dist',
             '2nd_closest_to_0_dist_x_atomic_mass',
             '2nd_closest_to_0_valence',
             '2nd_closest_to_0_valence_x_cube_inv_dist',
             '2nd_closest_to_1_atomic_mass',
             '2nd_closest_to_1_atomic_mass_x_cube_inv_dist',
             '2nd_closest_to_1_dist_x_atomic_mass',
             '2nd_closest_to_1_valence',
             '2nd_closest_to_1_valence_x_cube_inv_dist',
             '3rd_closest_to_0_atomic_mass',
             '3rd_closest_to_0_atomic_mass_x_cube_inv_dist',
             '3rd_closest_to_0_dist_x_atomic_mass',
             '3rd_closest_to_0_valence',
             '3rd_closest_to_0_valence_x_cube_inv_dist',
             '3rd_closest_to_1_atomic_mass',
             '3rd_closest_to_1_atomic_mass_x_cube_inv_dist',
             '3rd_closest_to_1_dist_x_atomic_mass',
             '3rd_closest_to_1_valence',
             '3rd_closest_to_1_valence_x_cube_inv_dist',
             '4th_closest_to_0_atomic_mass',
             '4th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '4th_closest_to_0_dist_x_atomic_mass',
             '4th_closest_to_0_valence',
             '4th_closest_to_0_valence_x_cube_inv_dist',
             '4th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '4th_closest_to_1_dist_x_atomic_mass',
             '4th_closest_to_1_valence',
             '4th_closest_to_1_valence_x_cube_inv_dist',
             '5th_closest_to_0_atomic_mass',
             '5th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '5th_closest_to_0_dist_x_atomic_mass',
             '5th_closest_to_0_valence',
             '5th_closest_to_0_valence_x_cube_inv_dist',
             '5th_closest_to_1_atomic_mass',
             '5th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '5th_closest_to_1_dist_x_atomic_mass',
             '5th_closest_to_1_valence',
             '5th_closest_to_1_valence_x_cube_inv_dist',
             '6th_closest_to_0_atomic_mass',
             '6th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '6th_closest_to_0_dist_x_atomic_mass',
             '6th_closest_to_0_valence',
             '6th_closest_to_0_valence_x_cube_inv_dist',
             '6th_closest_to_1_atomic_mass',
             '6th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '6th_closest_to_1_dist_x_atomic_mass',
             '6th_closest_to_1_valence_x_cube_inv_dist',
             '7th_closest_to_0_atomic_mass',
             '7th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '7th_closest_to_0_dist_x_atomic_mass',
             '7th_closest_to_0_valence_x_cube_inv_dist',
             '7th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '7th_closest_to_1_dist_x_atomic_mass',
             '7th_closest_to_1_valence',
             '7th_closest_to_1_valence_x_cube_inv_dist',
             '8th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '8th_closest_to_0_dist_x_atomic_mass',
             '8th_closest_to_0_valence_x_cube_inv_dist',
             '8th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '8th_closest_to_1_dist_x_atomic_mass',
             '8th_closest_to_1_valence_x_cube_inv_dist',
             '9th_closest_to_0_atomic_mass',
             '9th_closest_to_0_atomic_mass_x_cube_inv_dist',
             '9th_closest_to_0_dist_x_atomic_mass',
             '9th_closest_to_0_valence_x_cube_inv_dist',
             '9th_closest_to_1_atomic_mass_x_cube_inv_dist',
             '9th_closest_to_1_dist_x_atomic_mass',
             '9th_closest_to_1_valence_x_cube_inv_dist',
             'N1',
             'N2',
             'NH',
             'adC1',
             'adC2',
             'adC3',
             'adC4',
             'adH1',
             'adH2',
             'adN1',
             'adN2',
             'angle_0_10th0_1',
             'angle_0_10th1_1',
             'angle_0_1_2nd1',
             'angle_0_1_3rd1',
             'angle_0_1_4th1',
             'angle_0_1_5th1',
             'angle_0_1_closest1',
             'angle_0_2nd0_1',
             'angle_0_3rd0_1',
             'angle_0_3rd1_1',
             'angle_0_4th0_1',
             'angle_0_4th1_1',
             'angle_0_5th0_1',
             'angle_0_5th1_1',
             'angle_0_6th0_1',
             'angle_0_6th1_1',
             'angle_0_7th0_1',
             'angle_0_7th1_1',
             'angle_0_8th0_1',
             'angle_0_8th1_1',
             'angle_0_9th0_1',
             'angle_0_9th1_1',
             'angle_0_closest1_1',
             'angle_1_0_10th0',
             'angle_1_0_10th1',
             'angle_1_0_11th0',
             'angle_1_0_11th1',
             'angle_1_0_2nd0',
             'angle_1_0_3rd0',
             'angle_1_0_4th0',
             'angle_1_0_5th0',
             'angle_1_0_6th0',
             'angle_1_0_6th1',
             'angle_1_0_7th0',
             'angle_1_0_7th1',
             'angle_1_0_8th0',
             'angle_1_0_8th1',
             'angle_1_0_9th0',
             'angle_1_0_9th1',
             'angle_clos_0_2nd',
             'angle_clos_1_2nd',
             'atom1_valence',
             'atomic_mass_not_0_max',
             'atomic_mass_not_1_max',
             'atomic_mass_not_1_mean',
             'atomic_mass_not_1_std',
             'closest_to_0_atomic_mass_x_cube_inv_dist',
             'closest_to_0_dist_x_atomic_mass',
             'closest_to_0_spin_multiplicity',
             'closest_to_0_valence',
             'closest_to_0_valence_x_cube_inv_dist',
             'closest_to_1_atomic_mass_x_cube_inv_dist',
             'closest_to_1_dist_x_atomic_mass',
             'closest_to_1_valence_x_cube_inv_dist',
             'coulomb_H.x',
             'coulomb_H.y',
             'distC0',
             'distC1',
             'dist_to_0_max',
             'dist_to_0_min',
             'dist_to_1_max',
             'dist_to_1_min',
             'distance',
             'distance_10th_closest_to_0',
             'distance_10th_closest_to_1',
             'distance_10th_closest_to_1_cube_inverse',
             'distance_2nd_closest_to_0',
             'distance_2nd_closest_to_0_cube_inverse',
             'distance_2nd_closest_to_1',
             'distance_2nd_closest_to_1_cube_inverse',
             'distance_3rd_closest_to_0',
             'distance_3rd_closest_to_0_cube_inverse',
             'distance_3rd_closest_to_1',
             'distance_3rd_closest_to_1_cube_inverse',
             'distance_4th_closest_to_0',
             'distance_4th_closest_to_0_cube_inverse',
             'distance_4th_closest_to_1',
             'distance_4th_closest_to_1_cube_inverse',
             'distance_5th_closest_to_0',
             'distance_5th_closest_to_0_cube_inverse',
             'distance_5th_closest_to_1',
             'distance_5th_closest_to_1_cube_inverse',
             'distance_6th_closest_to_0',
             'distance_6th_closest_to_0_cube_inverse',
             'distance_6th_closest_to_1',
             'distance_6th_closest_to_1_cube_inverse',
             'distance_7th_closest_to_1',
             'distance_7th_closest_to_1_cube_inverse',
             'distance_8th_closest_to_1',
             'distance_8th_closest_to_1_cube_inverse',
             'distance_9th_closest_to_0',
             'distance_9th_closest_to_1',
             'distance_9th_closest_to_1_cube_inverse',
             'distance_closest_to_0',
             'distance_closest_to_0_cube_inverse',
             'distance_closest_to_1',
             'distance_closest_to_1_cube_inverse',
             'feat_acsf_g2_C_[1, 2]_atom0',
             'feat_acsf_g2_C_[1, 2]_atom1',
             'feat_acsf_g2_C_[1, 6]_atom0',
             'feat_acsf_g2_C_[1, 6]_atom1',
             'feat_acsf_g2_F_[0.01, 6]_atom1',
             'feat_acsf_g2_H_[0.01, 2]_atom1',
             'feat_acsf_g2_H_[0.01, 6]_atom1',
             'feat_acsf_g2_H_[0.1, 2]_atom1',
             'feat_acsf_g2_H_[0.1, 6]_atom1',
             'feat_acsf_g2_H_[1, 2]_atom0',
             'feat_acsf_g2_H_[1, 2]_atom1',
             'feat_acsf_g2_H_[1, 6]_atom0',
             'feat_acsf_g2_H_[1, 6]_atom1',
             'feat_acsf_g2_N_[0.01, 2]_atom0',
             'feat_acsf_g2_N_[1, 2]_atom1',
             'feat_acsf_g2_N_[1, 6]_atom1',
             'feat_acsf_g2_O_[1, 2]_atom0',
             'feat_acsf_g2_O_[1, 2]_atom1',
             'feat_acsf_g2_O_[1, 6]_atom0',
             'feat_acsf_g2_O_[1, 6]_atom1',
             'feat_acsf_g4_C_C_[0.01, 4, -1]_atom0',
             'feat_acsf_g4_C_C_[0.01, 4, -1]_atom1',
             'feat_acsf_g4_C_C_[0.01, 4, 1]_atom0',
             'feat_acsf_g4_C_C_[0.01, 4, 1]_atom1',
             'feat_acsf_g4_C_C_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_C_C_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_C_C_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_C_C_[1, 4, -1]_atom0',
             'feat_acsf_g4_C_C_[1, 4, -1]_atom1',
             'feat_acsf_g4_C_C_[1, 4, 1]_atom0',
             'feat_acsf_g4_C_C_[1, 4, 1]_atom1',
             'feat_acsf_g4_C_H_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_C_H_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_C_H_[0.1, 4, 1]_atom0',
             'feat_acsf_g4_C_H_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_C_H_[1, 4, -1]_atom0',
             'feat_acsf_g4_C_H_[1, 4, -1]_atom1',
             'feat_acsf_g4_C_H_[1, 4, 1]_atom0',
             'feat_acsf_g4_C_H_[1, 4, 1]_atom1',
             'feat_acsf_g4_H_H_[0.01, 4, -1]_atom0',
             'feat_acsf_g4_H_H_[0.01, 4, -1]_atom1',
             'feat_acsf_g4_H_H_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_H_H_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_H_H_[0.1, 4, 1]_atom0',
             'feat_acsf_g4_H_H_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_H_H_[1, 4, -1]_atom0',
             'feat_acsf_g4_H_H_[1, 4, -1]_atom1',
             'feat_acsf_g4_H_H_[1, 4, 1]_atom0',
             'feat_acsf_g4_H_H_[1, 4, 1]_atom1',
             'feat_acsf_g4_N_C_[0.01, 4, -1]_atom0',
             'feat_acsf_g4_N_C_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_N_C_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_N_C_[0.1, 4, 1]_atom0',
             'feat_acsf_g4_N_C_[1, 4, -1]_atom0',
             'feat_acsf_g4_N_C_[1, 4, -1]_atom1',
             'feat_acsf_g4_N_C_[1, 4, 1]_atom0',
             'feat_acsf_g4_N_C_[1, 4, 1]_atom1',
             'feat_acsf_g4_N_H_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_N_H_[1, 4, -1]_atom0',
             'feat_acsf_g4_N_H_[1, 4, -1]_atom1',
             'feat_acsf_g4_N_H_[1, 4, 1]_atom0',
             'feat_acsf_g4_N_H_[1, 4, 1]_atom1',
             'feat_acsf_g4_N_N_[0.01, 4, 1]_atom0',
             'feat_acsf_g4_N_N_[1, 4, -1]_atom1',
             'feat_acsf_g4_O_C_[0.01, 4, -1]_atom0',
             'feat_acsf_g4_O_C_[0.01, 4, -1]_atom1',
             'feat_acsf_g4_O_C_[0.01, 4, 1]_atom0',
             'feat_acsf_g4_O_C_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_O_C_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_O_C_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_O_C_[1, 4, -1]_atom0',
             'feat_acsf_g4_O_C_[1, 4, -1]_atom1',
             'feat_acsf_g4_O_C_[1, 4, 1]_atom0',
             'feat_acsf_g4_O_C_[1, 4, 1]_atom1',
             'feat_acsf_g4_O_H_[0.01, 4, -1]_atom1',
             'feat_acsf_g4_O_H_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_O_H_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_O_H_[0.1, 4, 1]_atom0',
             'feat_acsf_g4_O_H_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_O_H_[1, 4, -1]_atom0',
             'feat_acsf_g4_O_H_[1, 4, -1]_atom1',
             'feat_acsf_g4_O_H_[1, 4, 1]_atom0',
             'feat_acsf_g4_O_H_[1, 4, 1]_atom1',
             'feat_acsf_g4_O_N_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_O_N_[0.1, 4, 1]_atom1',
             'feat_acsf_g4_O_N_[1, 4, -1]_atom0',
             'feat_acsf_g4_O_N_[1, 4, -1]_atom1',
             'feat_acsf_g4_O_N_[1, 4, 1]_atom0',
             'feat_acsf_g4_O_O_[0.1, 4, -1]_atom0',
             'feat_acsf_g4_O_O_[0.1, 4, -1]_atom1',
             'feat_acsf_g4_O_O_[1, 4, -1]_atom0',
             'feat_acsf_g4_O_O_[1, 4, -1]_atom1',
             'inv_dist0E',
             'inv_dist0R',
             'inv_dist1',
             'inv_dist1E',
             'inv_dist1R',
             'inv_distP',
             'inv_distPE',
             'inv_distPR',
             'linkM0',
             'linkM1',
             'linkN',
             'max_molecule_atom_0_dist_xyz',
             'max_molecule_atom_1_dist_xyz',
             'mean_molecule_atom_0_dist_xyz',
             'mean_molecule_atom_1_dist_xyz',
             'min_molecule_atom_0_dist_xyz',
             'min_molecule_atom_1_dist_xyz',
             'sd_molecule_atom_1_dist_xyz',
             'tor_ang_2leftleft_count',
             'tor_ang_2leftleft_max',
             'tor_ang_2leftleft_mean',
             'tor_ang_2leftleft_min',
             'val_not_0_mean',
             'val_not_0_min',
             'val_not_0_std',
             'val_not_1_mean',
             'val_not_1_std',
             'yukawa_C.x',
             'yukawa_H.x',
             'yukawa_H.y',
             'yukawa_N.x',
             'yukawa_O.x'
]

# MODEL NUMBER
MODEL_NUMBER = 'TEST'
# script_name = os.path.basename(__file__).split('.')[0]
# if script_name not in MODEL_NUMBER:
#     logger.error('Model Number is not same as script! Update before running')
#     raise SystemExit('Model Number is not same as script! Update before running')

# Make a runid that is unique to the time this is run for easy tracking later
run_id = "{:%m%d_%H%M}".format(datetime.now())
LEARNING_RATE = 0.1
RUN_SINGLE_FOLD = False # Fold number to run starting with 1 - Set to False to run all folds
TARGET = 'scalar_coupling_constant'
N_ESTIMATORS = 500000
VERBOSE = 1000
EARLY_STOPPING_ROUNDS = 500
RANDOM_STATE = 529
N_THREADS = 48
N_FOLDS = 2
#EVAL_METRIC = 'group_mae'
EVAL_METRIC = 'MAE'
MODEL_TYPE = 'catboost'

In [3]:
#####################
## SETUP LOGGER
#####################
def get_logger():
    '''
        credits to: https://www.kaggle.com/ogrellier/user-level-lightgbm-lb-1-4480
    '''
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()
    FORMAT = '[%(levelname)s]%(asctime)s:%(name)s:%(message)s'
    logging.basicConfig(format=FORMAT)
    logger = logging.getLogger('main')
    logger.setLevel(logging.DEBUG)
#     handler = logging.StreamHandler(sys.stdout)
#     formatter = logging.Formatter(FORMAT)
#     handler.setFormatter(formatter)
#     logger.addHandler(handler)
    return logger
logger = get_logger()

In [ ]:
# Setup arrays for storing results
train_df = pd.read_parquet('../data/FE008_train.parquet') # only loading for skeleton not features
oof_df = train_df[['id', 'type','scalar_coupling_constant']].copy()
mol_group = train_df[['molecule_name','type']].copy()
del train_df
gc.collect()

oof_df['oof_preds'] = 0
test_df = pd.read_parquet('../data/FE008_test.parquet') # only loading for skeleton not features
prediction = np.zeros(len(test_df))
feature_importance = pd.DataFrame()
test_pred_df = test_df[['id','type','molecule_name']].copy()
del test_df
gc.collect()
test_pred_df['prediction'] = 0
bond_count = 1

folds = GroupKFold(n_splits=N_FOLDS)

In [5]:
def update_tracking(run_id, field, value, csv_file='tracking/tracking.csv',
                    integer=False, digits=None):
    print(run_id, field, value)

In [18]:
Meta_train.iloc[[5, 6]]['fc']

10    30.6116
14   -11.8159
Name: fc, dtype: float64

In [96]:
X_test['']

,10th_closest_to_0_atomic_mass_x_cube_inv_dist,10th_closest_to_0_valence_x_cube_inv_dist,10th_closest_to_1_atomic_mass_x_cube_inv_dist,10th_closest_to_1_valence_x_cube_inv_dist,12th_closest_to_0_atomic_mass_x_cube_inv_dist,12th_closest_to_1_atomic_mass_x_cube_inv_dist,13th_closest_to_1_atomic_mass_x_cube_inv_dist,14th_closest_to_1_atomic_mass_x_cube_inv_dist,21st_closest_to_0_atomic_mass_x_cube_inv_dist,2nd_closest_to_0_atomic_mass,...,val_not_0_min,val_not_0_std,val_not_1_mean,val_not_1_std,yukawa_C.x,yukawa_H.x,yukawa_H.y,yukawa_N.x,yukawa_O.x,meta_fc
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.01070,...,1.0,0.577350,1.333333,0.577350,0.449957,0.036035,0.449957,0.000000,0.000000,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.01070,...,1.0,0.577350,1.333333,0.577350,0.449957,0.036035,0.449957,0.000000,0.000000,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.403435,0.495676,1.180449,0.000000,0.124334,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.403465,0.495746,1.180449,0.000000,0.124334,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.373549,0.406458,1.180449,0.000000,0.132176,0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.403439,0.495688,1.180449,0.000000,0.124335,0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.403461,0.495734,1.180449,0.000000,0.124334,0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.356203,1.500000,1.069045,0.373549,0.406457,1.180449,0.000000,0.132176,0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.552648,1.750000,1.388730,0.552402,0.410170,1.104775,0.000000,0.000000,0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00794,...,1.0,1.552648,1.750000,1.388730,0.552403,0.410171,1.104775,0.000000,0.000000,0


In [104]:
DEPTH = 7

def fit_meta_feature(X_train, X_valid, X_test, Meta_train, train_idx, feature='fc', N_META_FOLDS=2, N_ESTIMATORS=5000):
    """
    Adds meta features to train, test and val
    """
    logger.info('X_train, X_valid and X_test are shapes {} {} {}'.format(X_train.shape, X_valid.shape, X_test.shape))

    logger.info(f'Creating meta feature {feature}')
    folds = GroupKFold(n_splits=N_META_FOLDS)
    fold_count = 1
    
    # Init predictions
    X_valid['meta_'+feature] = 0
    X_test['meta_'+feature] = 0
    X_train['meta_'+feature] = 0
    X_train_oof = X_train[['meta_'+feature]].copy()
    X_train = X_train.drop('meta_'+feature, axis=1)
    
    for fold_n, (train_idx2, valid_idx2) in enumerate(folds.split(X_train, groups=mol_group_type.iloc[train_idx].values)):
        logger.info('Running Meta Feature Type {} - Fold {} of {}'.format(feature, fold_count, folds.n_splits))

        logger.info(train_idx2)
        logger.info(valid_idx2)

        X_train2 = X_train.loc[X_train.reset_index().index.isin(train_idx2)]
        X_valid2 = X_train.loc[X_train.reset_index().index.isin(valid_idx2)]
        X_train2 = X_train2.copy()
        X_valid2 = X_valid2.copy()
        logger.info(X_train.shape)
        logger.info(X_train2.shape)
        logger.info(X_valid2.shape)

        y_train2 = Meta_train.loc[Meta_train.reset_index().index.isin(train_idx2)][feature]
        y_valid2 = Meta_train.loc[Meta_train.reset_index().index.isin(valid_idx2)][feature]
        fold_count += 1

        train_dataset = Pool(data=X_train2, label=y_train2)
        metavalid_dataset = Pool(data=X_valid2, label=y_valid2)
        valid_dataset = Pool(data=X_valid)
        test_dataset = Pool(data=X_test)
        model = CatBoostRegressor(iterations=N_ESTIMATORS,
                                     learning_rate=LEARNING_RATE,
                                     depth=DEPTH,
                                     eval_metric=EVAL_METRIC,
                                     verbose=VERBOSE,
                                     random_state = RANDOM_STATE,
                                     thread_count=N_THREADS,
                                     # loss_function=EVAL_METRIC,
                                     # bootstrap_type='Poisson',
                                     # bagging_temperature=5,
                                     task_type = "GPU") # Train on GPU

        model.fit(train_dataset,
                  eval_set=metavalid_dataset,
                  early_stopping_rounds=EARLY_STOPPING_ROUNDS)
        y_pred_meta_valid = model.predict(metavalid_dataset)
        y_pred_valid = model.predict(valid_dataset)
        y_pred = model.predict(test_dataset)

        X_train_oof.loc[X_train_oof.reset_index().index.isin(valid_idx2), 'meta_'+feature] = y_pred_meta_valid
        X_valid['meta_'+feature] += y_pred_valid
        X_test['meta_'+feature] += y_pred

    X_valid['meta_'+feature] = X_valid['meta_'+feature] / N_META_FOLDS
    X_test['meta_'+feature] = X_test['meta_'+feature] / N_META_FOLDS
    X_train['meta_'+feature] = X_train_oof['meta_'+feature]
    logger.info('Done creating meta features')
    logger.info('X_train, X_valid and X_test are shapes {} {} {}'.format(X_train.shape, X_valid.shape, X_test.shape))
    return X_train, X_valid, X_test

In [105]:
# scc = pd.read_csv('../input/scalar_coupling_contributions.csv')
# train = pd.read_csv('../input/train.csv')
# tr_scc = pd.merge(train, scc, on=['molecule_name','atom_index_0','atom_index_1','type'])

In [106]:
# tr_scc.to_parquet('../data/tr_scc.parquet')

In [107]:
tr_scc = pd.read_parquet('../data/tr_scc.parquet')

In [108]:
types = ['1JHC']

for bond_type in types:
    # Read the files and make X, X_test, and y
    train_df = pd.read_parquet('../data/FE018/FE018-train-{}.parquet'.format(bond_type))
    test_df = pd.read_parquet('../data/FE018/FE018-test-{}.parquet'.format(bond_type))
    Meta = train_df[['molecule_name', 'atom_index_0', 'atom_index_1']].merge(
        tr_cc, on=['molecule_name', 'atom_index_0', 'atom_index_1'])[['fc','sd','pso','dso']]
    X_type = train_df[FEATURES].copy()
    X_test_type = test_df[FEATURES].copy()
    y_type = train_df[TARGET].copy()
    del train_df
    del test_df
    gc.collect()
    # Remove colmns that have all nulls
    logger.info('{} Features before dropping null columns'.format(len(X_type.columns)))
    X_type = X_type.dropna(how='all', axis=1)
    X_test_type = X_test_type[X_type.columns]
    logger.info('{} Features after dropping null columns'.format(len(X_type.columns)))
    # Start training for type
    bond_start = timer()
    fold_count = 0 # Will be incremented at the start of the fold
    # Train the model
    # X_type = X.loc[X['type'] == bond_type]
    # y_type = y.iloc[X_type.index]
    # X_test_type = X_test.loc[X_test['type'] == bond_type]
    mol_group_type = mol_group.loc[mol_group['type'] == bond_type]['molecule_name']
    oof = np.zeros(len(X_type))
    prediction_type = np.zeros(len(X_test_type))
    bond_scores = []
    for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X_type, groups=mol_group_type)):
        Meta_train, Meta_valid = Meta.iloc[train_idx], Meta.iloc[valid_idx]
        fold_count += 1 # First fold is 1
        if RUN_SINGLE_FOLD is not False:
            if fold_count != RUN_SINGLE_FOLD:
                logger.info('Running only for fold {}, skipping fold {}'.format(RUN_SINGLE_FOLD, fold_count))
                continue
        if MODEL_TYPE == 'lgbm':
            fold_start = timer()
            logger.info('Running Type {} - Fold {} of {}'.format(bond_type,
                                                           fold_count, folds.n_splits))
            X_train, X_valid = X_type.iloc[train_idx], X_type.iloc[valid_idx]
            y_train, y_valid = y_type.iloc[train_idx], y_type.iloc[valid_idx]
            X_train, X_valid, X_test_type = fit_meta_feature(X_train, X_valid, X_test_type, Meta_train, train_idx)
            model = lgb.LGBMRegressor(**lgb_params, n_estimators=N_ESTIMATORS, n_jobs=N_THREADS)
            model.fit(X_train, y_train,
                      eval_set=[#(X_train, y_train),
                                (X_valid, y_valid)],
                      eval_metric=EVAL_METRIC,
                      verbose=VERBOSE,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS)
            now = timer()
            update_tracking(run_id, '{}_tr_sec_f{}'.format(bond_type, fold_n+1), (now-fold_start), integer=True)
            logger.info('Saving model file')
            model.booster_.save_model('models/{}-{}-{}-{}.model'.format(MODEL_NUMBER,
                                                               run_id,
                                                               bond_type,
                                                               fold_count))
            pred_start = timer()
            logger.info('Predicting on validation set')
            y_pred_valid = model.predict(X_valid,
                                         num_iteration=model.best_iteration_,
                                         n_jobs=N_THREADS)
            logger.info('Predicting on test set')
            y_pred = model.predict(X_test_type,
                                   num_iteration=model.best_iteration_)
            now = timer()
            update_tracking(run_id, '{}_pred_sec_f{}'.format(bond_type, fold_n+1), (now-pred_start), integer=True)
            update_tracking(run_id, '{}_f{}_best_iter'.format(bond_type, fold_n+1), model.best_iteration_, integer=True)

            # feature importance
            logger.info('Storing the fold importance')
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X_train.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["type"] = bond_type
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat(
                [feature_importance, fold_importance], axis=0)

            bond_scores.append(mean_absolute_error(y_valid, y_pred_valid))
            logger.info('CV mean score: {0:.4f}, std: {1:.4f}.'.format(
                np.mean(bond_scores), np.std(bond_scores)))
            oof[valid_idx] = y_pred_valid.reshape(-1,)
            prediction_type += y_pred
        elif MODEL_TYPE == 'catboost':
            fold_start = timer()
            logger.info('Running Type {} - Fold {} of {}'.format(bond_type,
                                                           fold_count, folds.n_splits))
            X_train, X_valid = X_type.iloc[train_idx], X_type.iloc[valid_idx]
            X_train = X_train.copy()
            X_valid = X_valid.copy()
            y_train, y_valid = y_type.iloc[train_idx], y_type.iloc[valid_idx]

            ### ADD META FEATURES
            X_train, X_valid, X_test_type = fit_meta_feature(X_train, X_valid, X_test_type, Meta_train, train_idx)

            DEPTH = 7
            update_tracking(run_id, 'depth', DEPTH)
            train_dataset = Pool(data=X_train, label=y_train)
            valid_dataset = Pool(data=X_valid, label=y_valid)
            test_dataset = Pool(data=X_test_type)
            model = CatBoostRegressor(iterations=N_ESTIMATORS,
                                         learning_rate=LEARNING_RATE,
                                         depth=DEPTH,
                                         eval_metric=EVAL_METRIC,
                                         verbose=VERBOSE,
                                         random_state = RANDOM_STATE,
                                         thread_count=N_THREADS,
                                         # loss_function=EVAL_METRIC,
                                         # bootstrap_type='Poisson',
                                         # bagging_temperature=5,
                                         task_type = "GPU") # Train on GPU

            model.fit(train_dataset,
                      eval_set=valid_dataset,
                      early_stopping_rounds=EARLY_STOPPING_ROUNDS)
            now = timer()
            update_tracking(run_id, '{}_tr_sec_f{}'.format(bond_type, fold_n+1), (now-fold_start), integer=True)
            logger.info('Saving model file')
            model.save_model('models/{}/{}-{}-{}-{}.model'.format(MODEL_NUMBER,
                                                                  MODEL_NUMBER,
                                                               run_id,
                                                               bond_type,
                                                               fold_count))
            pred_start = timer()
            logger.info('Predicting on validation set')
            y_pred_valid = model.predict(valid_dataset)
            logger.info('Predicting on test set')
            y_pred = model.predict(test_dataset)
            now = timer()
            update_tracking(run_id, '{}_pred_sec_f{}'.format(bond_type, fold_n+1), (now-pred_start), integer=True)
            update_tracking(run_id, '{}_f{}_best_iter'.format(bond_type, fold_n+1), model.best_iteration_, integer=True)

            # feature importance
            logger.info('Storing the fold importance')
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = model.feature_names_
            fold_importance["importance"] = model.get_feature_importance()
            fold_importance["type"] = bond_type
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat(
                [feature_importance, fold_importance], axis=0)
            fold_score = mean_absolute_error(y_valid, y_pred_valid)
            bond_scores.append(fold_score)
            update_tracking(run_id, '{}cv_f{}'.format(bond_type, fold_n+1), fold_score, integer=False)
            logger.info('CV mean score: {0:.4f}, std: {1:.4f}.'.format(
                np.mean(bond_scores), np.std(bond_scores)))
            oof[valid_idx] = y_pred_valid.reshape(-1,)
            prediction_type += y_pred
        now = timer()
        logger.info('Completed training and predicting for bond {} fold {}-of-{} in {:0.4f} seconds'.format(bond_type,
                                                                                                            fold_count,
                                                                                                            N_FOLDS,
                                                                                                            now-fold_start))
    update_tracking(run_id, f'{bond_type}_mae_cv', np.mean(bond_scores), digits=4)
    update_tracking(run_id, f'{bond_type}_std_mae_cv', np.std(bond_scores), digits=6)
    oof_df.loc[oof_df['type'] == bond_type, 'oof_preds'] = oof
    if RUN_SINGLE_FOLD is False:
        prediction_type /= folds.n_splits
    test_pred_df.loc[test_pred_df['type'] ==
                     bond_type, 'prediction'] = prediction_type

    pred_pq_name = '{}_{}_sub_{:0.4f}_{}_{}folds_{}iter_{}lr.parquet'.format(MODEL_NUMBER, run_id, np.mean(bond_scores),
                                                                               MODEL_TYPE, N_FOLDS, N_ESTIMATORS, LEARNING_RATE)
    test_pred_df.to_parquet('type_results/{}/{}'.format(bond_type, pred_pq_name))
    oof_pq_name = '{}_{}_oof_{:0.4f}_{}_{}folds_{}iter_{}lr.parquet'.format(MODEL_NUMBER, run_id, np.mean(bond_scores),
                                                                               MODEL_TYPE, N_FOLDS, N_ESTIMATORS, LEARNING_RATE)
    oof_df.loc[oof_df['type'] == bond_type].to_parquet('type_results/{}/{}'.format(bond_type, oof_pq_name))
    now = timer()
    logger.info('Completed training and predicting for bond {} in {:0.4f} seconds'.format(bond_type,
                                                                                          now-bond_start))
    if bond_count != number_of_bonds:
        csv_save_start = timer()
        # Save the results inbetween bond type because it takes a long time
        submission_csv_name = 'temp/{}/temp{}of{}_{}_{}_submission_{}_{}folds_{}iter_{}lr.csv'.format(
            MODEL_NUMBER, bond_count, number_of_bonds, MODEL_NUMBER, run_id, MODEL_TYPE, N_FOLDS, N_ESTIMATORS, LEARNING_RATE)
        oof_csv_name = 'temp/{}/temp{}of{}_{}_{}_oof_{}_{}folds_{}iter_{}lr.csv'.format(
            MODEL_NUMBER, bond_count, number_of_bonds, MODEL_NUMBER, run_id, MODEL_TYPE, N_FOLDS, N_ESTIMATORS, LEARNING_RATE)
        fi_csv_name = 'temp/{}/temp{}of{}_{}_{}_fi_{}_{}folds_{}iter_{}lr.csv'.format(
                MODEL_NUMBER, bond_count, number_of_bonds, MODEL_NUMBER, run_id, MODEL_TYPE, N_FOLDS, N_ESTIMATORS, LEARNING_RATE)

        logger.info('Saving Temporary LGB Submission files:')
        logger.info(submission_csv_name)
        ss = pd.read_csv('input/sample_submission.csv')
        ss['scalar_coupling_constant'] = test_pred_df['prediction']
        ss.to_csv(submission_csv_name, index=False)
        # OOF
        oof_df.to_csv(oof_csv_name, index=False)
        # Feature Importance
        feature_importance.to_csv(fi_csv_name, index=False)
        now = timer()
        update_tracking(run_id, '{}_csv_save_sec'.format(bond_type), (now-csv_save_start), integer=True)
    bond_count += 1
oof_score = mean_absolute_error(
    oof_df['scalar_coupling_constant'], oof_df['oof_preds'])
update_tracking(run_id, 'oof_score', oof_score, digits=4)
logger.info('Out of fold score is {:.4f}'.format(oof_score))

oof_gml_score = group_mean_log_mae(
    oof_df['scalar_coupling_constant'], oof_df['oof_preds'], oof_df['type'])
update_tracking(run_id, 'gml_oof_score', oof_gml_score, digits=4)
logger.info('Out of fold group mean log mae score is {:.4f}'.format(oof_gml_score))

[INFO]2019-07-12 19:17:21,672:main:310 Features before dropping null columns
[INFO]2019-07-12 19:17:25,051:main:310 Features after dropping null columns
[INFO]2019-07-12 19:17:25,826:main:Running Type 1JHC - Fold 1 of 2
[INFO]2019-07-12 19:17:26,744:main:X_train, X_valid and X_test are shapes (354708, 310) (354708, 310) (380609, 310)
[INFO]2019-07-12 19:17:26,745:main:Creating meta feature fc
[INFO]2019-07-12 19:17:27,279:main:Running Meta Feature Type fc - Fold 1 of 2
[INFO]2019-07-12 19:17:27,279:main:[     6      7      8 ... 354705 354706 354707]
[INFO]2019-07-12 19:17:27,280:main:[     0      1      2 ... 354696 354697 354698]
[INFO]2019-07-12 19:17:27,968:main:(354708, 310)
[INFO]2019-07-12 19:17:27,969:main:(177354, 310)
[INFO]2019-07-12 19:17:27,969:main:(177354, 310)


0:	learn: 84.1448177	test: 84.1037981	best: 84.1037981 (0)	total: 9.31ms	remaining: 46.6s
1000:	learn: 0.9699804	test: 1.0669659	best: 1.0669659 (1000)	total: 7.59s	remaining: 30.3s
2000:	learn: 0.7786249	test: 0.9311679	best: 0.9311679 (2000)	total: 15.1s	remaining: 22.6s
3000:	learn: 0.6684446	test: 0.8650097	best: 0.8650097 (3000)	total: 22.7s	remaining: 15.1s
4000:	learn: 0.5900510	test: 0.8228064	best: 0.8228064 (4000)	total: 30.3s	remaining: 7.56s
4999:	learn: 0.5298528	test: 0.7937319	best: 0.7937319 (4999)	total: 38s	remaining: 0us
bestTest = 0.7937319217
bestIteration = 4999
Shrink model to first 5000 iterations.


[INFO]2019-07-12 19:18:48,971:main:Running Meta Feature Type fc - Fold 2 of 2
[INFO]2019-07-12 19:18:48,972:main:[     0      1      2 ... 354696 354697 354698]
[INFO]2019-07-12 19:18:48,972:main:[     6      7      8 ... 354705 354706 354707]
[INFO]2019-07-12 19:18:49,785:main:(354708, 310)
[INFO]2019-07-12 19:18:49,786:main:(177354, 310)
[INFO]2019-07-12 19:18:49,787:main:(177354, 310)


0:	learn: 84.1072544	test: 84.1489902	best: 84.1489902 (0)	total: 8.79ms	remaining: 43.9s
1000:	learn: 0.9559653	test: 1.0657313	best: 1.0657313 (1000)	total: 7.51s	remaining: 30s
2000:	learn: 0.7659408	test: 0.9334419	best: 0.9334419 (2000)	total: 15s	remaining: 22.4s
3000:	learn: 0.6585454	test: 0.8697763	best: 0.8697763 (3000)	total: 22.5s	remaining: 15s
4000:	learn: 0.5827202	test: 0.8293808	best: 0.8293808 (4000)	total: 30.1s	remaining: 7.53s
4999:	learn: 0.5241064	test: 0.8015442	best: 0.8015442 (4999)	total: 37.9s	remaining: 0us
bestTest = 0.8015442455
bestIteration = 4999
Shrink model to first 5000 iterations.


[INFO]2019-07-12 19:20:11,659:main:Done creating meta features
[INFO]2019-07-12 19:20:11,660:main:X_train, X_valid and X_test are shapes (354708, 311) (354708, 311) (380609, 311)


0712_1816 depth 7
0:	learn: 85.4921457	test: 85.4773053	best: 85.4773053 (0)	total: 12.6ms	remaining: 1h 45m 17s
1000:	learn: 0.7748057	test: 0.7642397	best: 0.7642397 (1000)	total: 9.95s	remaining: 1h 22m 41s
2000:	learn: 0.7176167	test: 0.7320179	best: 0.7320179 (2000)	total: 20.3s	remaining: 1h 24m 5s
3000:	learn: 0.6703993	test: 0.7079702	best: 0.7079702 (3000)	total: 30.7s	remaining: 1h 24m 39s
4000:	learn: 0.6299007	test: 0.6879805	best: 0.6879805 (4000)	total: 41.1s	remaining: 1h 24m 58s
5000:	learn: 0.5945724	test: 0.6715435	best: 0.6715435 (5000)	total: 51.7s	remaining: 1h 25m 19s
6000:	learn: 0.5627298	test: 0.6571679	best: 0.6571679 (6000)	total: 1m 2s	remaining: 1h 25m 24s
7000:	learn: 0.5345530	test: 0.6449087	best: 0.6449087 (7000)	total: 1m 13s	remaining: 1h 25m 42s
8000:	learn: 0.5085149	test: 0.6339243	best: 0.6339243 (8000)	total: 1m 24s	remaining: 1h 26m 58s
9000:	learn: 0.4849133	test: 0.6243402	best: 0.6243393 (8999)	total: 1m 35s	remaining: 1h 26m 58s
10000:	learn

KeyboardInterrupt: 

In [111]:
model.best_iteration_